In [35]:
import pdfplumber
import re
import json
import random
import string

# 받침 여부 확인 함수
def has_jongseong(char):
    return (ord(char) - 0xAC00) % 28 != 0

# 조사 선택 함수
def get_particle(term, particle_type):
    term = term.strip()
    if not term or particle_type == 'none':
        return ''
    last_char = term[-1]
    particles = {'subject': ('이', '가'), 'object': ('을', '를')}
    return particles.get(particle_type, ('', ''))[0 if has_jongseong(last_char) else 1]

# 질문 패턴 리스트
question_patterns = [
    {"pattern": "{}", "particle_type": "none"},
    {"pattern": "{}{} 무엇인가요?", "particle_type": "subject"},
    {"pattern": "{}{} 뭐야?", "particle_type": "subject"},
    {"pattern": "{}{} 설명해줘.", "particle_type": "object"},
    {"pattern": "{}에 대해 알려줘.", "particle_type": "none"},
    {"pattern": "{}에 대해 설명해줄래?", "particle_type": "none"},
    {"pattern": "{}란?", "particle_type": "none"},
    {"pattern": "{}{} 무엇이니?", "particle_type": "subject"},
    {"pattern": "{}{} 알고 싶어.", "particle_type": "object"},
    {"pattern": "{}{} 자세히 설명해줘.", "particle_type": "object"},
    {"pattern": "{}의 뜻이 뭐야?", "particle_type": "none"},
    {"pattern": "{}에 대해 이야기해줘.", "particle_type": "none"},
    {"pattern": "{}{} 어떤 의미야?", "particle_type": "subject"},
    {"pattern": "{}에 대해 자세히 알고 싶어.", "particle_type": "none"},
    {"pattern": "{}의 정의는?", "particle_type": "none"},
    {"pattern": "{}에 대해 설명 부탁해.", "particle_type": "none"}
]

# 질문 생성 함수
def generate_question(term):
    pattern_info = random.choice(question_patterns)
    particle = get_particle(term, pattern_info["particle_type"])
    return pattern_info["pattern"].format(term.strip(), particle)

# 정의 클린업 함수
def clean_definition(definition):
    return re.sub(r'\s{2,}', ' ', re.sub(r'\s+([.,!?])', r'\1', re.sub(r'\s+\d+$', '', definition))).strip()

# PDF에서 용어와 설명 추출 함수
def extract_pdf(pdf_path):
    terms_definitions = {}
    term = definition = ""
    term_okay = False

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            words = page.extract_words(extra_attrs=["size"])
            for word_info in words:
                text = word_info['text'].strip()
                size = word_info['size']

                if 13.0 <= size < 13.1:
                    if term_okay and term.strip() and definition.strip():
                        terms_definitions[term.strip()] = clean_definition(definition)
                        term = definition = ""
                    term_okay = False
                    term += " " + text
                elif 10.9 <= size < 11.0:
                    term_okay = True
                    definition += " " + text

        if term_okay and term.strip() and definition.strip():
            terms_definitions[term.strip()] = clean_definition(definition)

    return terms_definitions

# 메인 코드
terms = extract_pdf('./data/finance_term.pdf')
items = list(terms.items())[1:]

qa_list = [
    {"질문": generate_question(term), "대답": definition.strip()}
    for term, definition in items if term.strip() and definition.strip()
]

# JSON 파일로 저장
with open('finance_terms_qa.json', 'w', encoding='utf-8') as f:
    json.dump(qa_list, f, ensure_ascii=False, indent=4)

# 일부 출력
for qa in qa_list[:5]:
    print(f"질문: {qa['질문']}\n대답: {qa['대답']}\n")

질문: 가계부실위험지수(HDRI)에 대해 이야기해줘.
대답: 가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상 환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA 가 각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다.

질문: 가계수지를 설명해줘.
대답: 가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 2 흑자를 냈다면 그 가정은 벌어들인 수입 일부만을 사용했다는 것을 의미하며, 적자를 냈다면 수입 외에 빚을 추가로 얻어 사용한 것이라고 보아야 한다. 우리나라는 통계청에 서 가계의 수입과 지출을 조사하여 국민의 소득수준 및 생활실태를 파악하기 위해 표본으로 선정된 가계에 가계부를 나누어 주고 한 달간의 소득과 지출을 기록하도록 한 다음 이를 토대로 가계수지 통계를 작성하여 발표하고 있다. 가계부의 소득항목에는 근로소득 사업소득 재산소득 이전소득 항목이 있고, 비용항목에는 식료품비 주거비 수도광열비 보건의료비 교육비 항목이 있다.

질문: 가계순저축률이 뭐야?
대답: 일반적으로 저축률은 저축액을 처분가능소득으로 나눈 비율을 말한다. 마찬가지로 가계순저축률은 가계부문